# Demo: Defining Control_M Workflows using Python

# Step 1 - Setup

## Step 1A - Install the library

In [1]:
!pip --version

pip 21.2.4 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/tadinve/naga.git

  Cloning https://github.com/tadinve/naga.git to /tmp/pip-req-build-7xnxlv_p
  Running command git clone -q https://github.com/tadinve/naga.git /tmp/pip-req-build-7xnxlv_p
  Resolved https://github.com/tadinve/naga.git to commit 2aca099b9186d2c1e6689cdf6321cad93031eff0
  Created wheel for ctm-python-client: filename=ctm_python_client-0.3.0-py3-none-any.whl size=615435 sha256=9e6aa32f9e26878bfd7456567ce22cf6b31c812c79c98b85250a894014282a3d
  Stored in directory: /tmp/pip-ephem-wheel-cache-wqynq2d9/wheels/de/35/cb/b70c2ffd6e40942c62635105854bc5fabd5125a0f0f9b17cb9
Successfully built ctm-python-client
  Attempting uninstall: ctm-python-client
    Found existing installation: ctm-python-client 0.3.0
    Uninstalling ctm-python-client-0.3.0:
      Successfully uninstalled ctm-python-client-0.3.0


In [3]:
from ctm_python_client.core.bmc_control_m import CmJobFlow
from ctm_python_client.jobs.dummy import DummyJob

# Step 2 - Instantiate, Authenticate and Schedule


## Step 2A - Create the object

In [4]:
# Please change the URfrI, and ctm_user and enter ctm_password to match your environment
from ctm_python_client.session.session import Session

import getpass
ctm_uri =  "https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api"
ctm_user =  "vtadinad"
ctm_pwd =                                                                                                                                               "P4ssw0rd"
if "ctm_pwd" not in locals(): # has not been enterd once, will skip next time
    ctm_pwd = getpass.getpass("Enter your Control M Password ")

session = Session( endpoint=ctm_uri,username=ctm_user,password=ctm_pwd)
session.get_token()

'892DB665F33EB30F948515B452A3496D661734A00CCE8DDEA1862F93574A0064FF53F02C3147F569908A201FA29E288934CC7502E509D35B6C510C4ACC447F8F'

In [5]:
t1_flow = CmJobFlow(application="Naga0.3_Examples",sub_application="Demo-OR_JOB",session=session)

## Step 2B - Define the Schedule

In [6]:
t1_flow.set_run_as(username="ctmuser",host="acb-rhctmv20")

In [7]:
# Define the schedule
months = ["JAN", "OCT", "DEC"]
monthDays = ["ALL"]
weekDays = ["MON","TUE", "WED", "THU", "FRI"]
fromTime = "0300"
toTime = "2100"
t1_flow.set_schedule(months, monthDays, weekDays,fromTime, toTime)


# Step 3  - Create Folder

In [8]:
# Create Fodler
f1 = t1_flow.create_folder(name="OR-JOB")

# Step 4 - Create Tasks

In [9]:
start = t1_flow.add_job( f1, DummyJob(f1,"Start-Flow") )


job1 = DummyJob(f1,"Job1")
job1.add_if_output("if-true", "*true*", "Job1-TO-Job2")
job1.add_if_output("if-flase", "*false*", "Job1-TO-Job3")
job1_id = t1_flow.add_job( f1, job1 )

job2 = DummyJob(f1,"Job2")
job2_id = t1_flow.add_job( f1, job2 )

job3 = DummyJob(f1,"Job3")
job3_id = t1_flow.add_job( f1, job3 )

job4 = DummyJob(f1,"Job4")
job4.wait_for_jobs(job1_id, job3_id, condition="OR")
job4_id = t1_flow.add_job( f1, job4 )

end = t1_flow.add_job(f1, DummyJob(f1,"End-Flow") )



# Step 5 - Chain Tasks

In [10]:
# start >>  hello_world_id >> end
t1_flow.chain_jobs(f1,[start, job1_id])
t1_flow.chain_jobs(f1,[job2_id, job3_id])
t1_flow.chain_jobs(f1,[job4_id, end])

[('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6af0>), ('Job1', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6a30>), ('Job2', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6790>), ('Job3', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6ac0>), ('Job4', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6b50>), ('End-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6760>)]
['0', '1']
[('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6af0>), ('Job1', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6a30>), ('Job2', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6790>), ('Job3', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6ac0>), ('Job4', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6b50>), ('End-Flow', <ctm_python_client.jobs.dummy.DummyJob object at 0x7f0939bb6760>)]
['2', '3']
[('Start-Flow', <ctm_python_client.jobs.dumm

# Step 6 - Display Workflow

## Step 6A - Display DAG

In [11]:
# View the t1_flow Details
nodes, edges = t1_flow.get_nodes_and_edges()
nodes, edges

([('Start-Flow', <ctm_python_client.jobs.dummy.DummyJob at 0x7f0939bb6af0>),
  ('Job1', <ctm_python_client.jobs.dummy.DummyJob at 0x7f0939bb6a30>),
  ('Job2', <ctm_python_client.jobs.dummy.DummyJob at 0x7f0939bb6790>),
  ('Job3', <ctm_python_client.jobs.dummy.DummyJob at 0x7f0939bb6ac0>),
  ('Job4', <ctm_python_client.jobs.dummy.DummyJob at 0x7f0939bb6b50>),
  ('End-Flow', <ctm_python_client.jobs.dummy.DummyJob at 0x7f0939bb6760>)],
 [('Start-Flow', 'Job1'), ('Job2', 'Job3'), ('Job4', 'End-Flow')])

In [12]:
#display using graphviz
from ctm_python_client.utils.displayDAG import DisplayDAG

#sudo apt-get install graphviz (on unix)
#  or 
# brew install graphviz (for mac)
#DisplayDAG(t1_flow).display_graphviz()

## Step 6B - Display JSON

In [13]:
t1_flow.display_json()

{
    "Defaults": {
        "Application": "Naga0.3_Examples",
        "SubApplication": "Demo-OR_JOB",
        "RunAs": "ctmuser",
        "Host": "acb-rhctmv20",
        "When": {
            "Months": [
                "JAN",
                "OCT",
                "DEC"
            ],
            "MonthDays": [
                "ALL"
            ],
            "WeekDays": [
                "MON",
                "TUE",
                "WED",
                "THU",
                "FRI"
            ],
            "FromTime": "0300",
            "ToTime": "2100"
        }
    },
    "OR-JOB": {
        "Type": "Folder",
        "Start-Flow": {
            "Type": "Job:Dummy",
            "eventsToAdd": {
                "Type": "AddEvents",
                "Events": [
                    {
                        "Event": "Start-Flow-To-Job1"
                    }
                ]
            }
        },
        "Job1": {
            "Type": "Job:Dummy",
            "if-true": {
    

# Step 7 - Submit Workflow to Control-M

In [14]:
t1_flow.deploy()

Error deploying job, look for more details below
(400)
Reason: 
HTTP response headers: HTTPHeaderDict({'X-Content-Type-Options': 'nosniff', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'no-referrer', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Cache-Control': 'no-cache, no-store, private, must-revalidate, max-age=0, no-transform', 'Content-Security-Policy': "object-src 'none'; base-uri 'self';plugin-types application/x-java-applet; sandbox allow-scripts allow-popups allow-forms allow-top-navigation allow-presentation allow-same-origin allow-downloads; report-uri /csp-violation-report-endpoint/; form-action 'self'; frame-ancestors 'none';", 'hstsMaxAgeSeconds': '31536000', 'Set-Cookie': 'SameSite=strict; Secure; HttpOnly, JSESSIONID=AC93B908E00787EC44E0F1682FDEE55F; Path=/automation-api; Secure; HttpOnly', 'X-XSS-Protection': '1; mode=block', 'Expect-CT': 'max-age=120, enforce', 'vary': 'Origin', 'Server': 'Web Server', 'Con

In [15]:
t1_flow.run()

Error deploying job, look for more details below
(400)
Reason: 
HTTP response headers: HTTPHeaderDict({'X-Content-Type-Options': 'nosniff', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'no-referrer', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Cache-Control': 'no-cache, no-store, private, must-revalidate, max-age=0, no-transform', 'Content-Security-Policy': "object-src 'none'; base-uri 'self';plugin-types application/x-java-applet; sandbox allow-scripts allow-popups allow-forms allow-top-navigation allow-presentation allow-same-origin allow-downloads; report-uri /csp-violation-report-endpoint/; form-action 'self'; frame-ancestors 'none';", 'hstsMaxAgeSeconds': '31536000', 'Set-Cookie': 'SameSite=strict; Secure; HttpOnly, JSESSIONID=ADD035C72CBD2022C5EF92E9E4140AA1; Path=/automation-api; Secure; HttpOnly', 'X-XSS-Protection': '1; mode=block', 'Expect-CT': 'max-age=120, enforce', 'vary': 'Origin', 'Server': 'Web Server', 'Con